##### Jupyter Notebook is available at : [Single Docking](https://github.com/MangalamGSinha/DockSuiteX/tree/main/docs/examples/single_docking.ipynb)

In [ ]:
from docksuitex import Protein, Ligand, PocketFinder, VinaDocking, AD4Docking
from docksuitex.utils import fetch_pdb, fetch_sdf, parse_vina_log_to_csv, parse_ad4_dlg_to_csv, view_molecule

### Fetch and prepare Protein

In [ ]:
# Fetch the protein structure (1HVR) from the PDB
Protein_path = fetch_pdb(pdbid = "1HVR")

# Visualize the downloaded protein structure
view_molecule(Protein_path)

In [ ]:
# Prepare the protein
# This cleans the PDB, removes water/heterogens, and adds hydrogens/charges

prot = Protein(
    input= Protein_path, 

    #Default parameters
    fix_pdb =True, 
    remove_heterogens=True, 
    remove_water=True,
    add_hydrogens=True, 
    add_charges=True, 
    preserve_charge_types=None,
    )

prot.prepare()

# Visualize the prepared protein structure
prot.view_molecule()

### Find Binding Pockets

In [ ]:
# Identify binding pockets on the protein surface using P2Rank
pf = PocketFinder(input="1HVR.pdbqt")
pockets = pf.run()

### Fetch and prepare Ligand

In [ ]:
# Fetch the ligand structure (CID 338) from PubChem
Ligand_path = fetch_sdf(cid = "338")

# Visualize the downloaded ligand structure
view_molecule(Ligand_path)

In [ ]:
# Prepare the ligand
# This minimizes the structure, removes water and adds hydrogens/charges

lig = Ligand(
    input=Ligand_path, 

    #Default parameters
    minimize=None,  #Options: "mmff94", "mmff94s", "uff", "gaff"
    remove_water=True, 
    add_hydrogens=True, 
    add_charges=True, 
    preserve_charge_types=None
)

lig.prepare()

# Visualize the prepared ligand structure
lig.view_molecule()

### Perform protein ligand docking at the first pocket

##### Using AutoDock Vina


In [ ]:
# Define the grid center based on the first identified pocket
center = pockets[0]['center']

# Initialize Vina docking with the receptor, ligand, and grid center
vina = VinaDocking(
    receptor="1HVR.pdbqt",
    ligand="338.pdbqt",
    grid_center=center,
    
    # Default Parameters
    grid_size=(20,20,20),
    exhaustiveness=8,
    num_modes=9,
    seed=42
)

# Run docking and save results
vina.run(save_to="vina_results/1HVR_338")

#Interactively view the docked poses
vina.view_results()

# Parse Vina results to a CSV summary
parse_vina_log_to_csv(results_dir="vina_results", output_csv="vina_results/vina_summary.csv")


#### Using AutoDock4

In [ ]:
# Define the grid center based on the first identified pocket
center = pockets[0]['center']

# Perform docking with the first pocket using AutoDock4
ad4 = AD4Docking(
    receptor="1HVR.pdbqt",
    ligand="338.pdbqt",
    grid_center=center,

    #Default Parameters
    grid_size=(60,60,60),
    spacing=0.375,     
    dielectric=-0.1465, 
    smooth=0.5,          
    ga_pop_size=150,
    ga_num_evals=2500000,
    ga_num_generations=27000,
    ga_elitism=1,
    ga_mutation_rate=0.02,
    ga_crossover_rate=0.8,
    ga_run=10,
    rmstol=2.0,
    seed=("pid", "time")
)


# Run docking and save results
ad4.run(save_to="ad4_results/1HVR_338")    

#Interactively view the docked poses
ad4.view_results()

# Parse AD4 results to a CSV summary
parse_ad4_dlg_to_csv(results_dir="ad4_results", output_csv="ad4_results/ad4_summary.csv")